In [1]:
!python -m pip install --user --upgrade pip

In [3]:
!pip3 install kfp --upgrade --user

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
     |████████████████████████████████| 159 kB 5.7 MB/s 
     |████████████████████████████████| 636 kB 16.4 MB/s 
     |████████████████████████████████| 1.5 MB 16.3 MB/s 
     |████████████████████████████████| 54 kB 2.7 MB/s 
     |████████████████████████████████| 50 kB 6.7 MB/s 
     |████████████████████████████████| 56 kB 4.5 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 87 kB 6.1 MB/s 
     |████████████████████████████████| 1.0 MB 29.6 MB/s 
     |████████████████████████████████| 61 kB 8.0 MB/s 
  Created wheel for kfp: filename=kfp-1.4.0-py3-none-any.whl size=221064 sha256=56bb488d9927c847eadff900e945a2f4af3ede7724dc68a752e96fc0ee862315
  Stored in directory: /root/.cac

In [2]:
import kfp
from kfp import dsl
import kfp.components as comp

In [3]:
def load_data_op():
    return dsl.ContainerOp(
        name = 'Load Data',
        image = 'mavencodevv/load_ld:v.0.1',
        arguments = [],
        file_outputs={
            'data': '/load_data/data'
        }      
    )

In [4]:
def preprocess_op(data):
    return dsl.ContainerOp(
        name = 'Preprocess Data',
        image = 'mavencodevv/preprocess_ld:v.0.3',
        arguments = ['--data', data],
        file_outputs={
            'clean_data': '/preprocessing/clean_data'   
        }
    )

In [5]:
def train_op(clean_data):
    return dsl.ContainerOp(
        name = 'train model',
        image = 'mavencodevv/train_ld:v.0.2',
        arguments = ['--clean_data', clean_data
            ],
        file_outputs={
            'fit_model': '/train_model/fit_model' 
        }
    )

In [6]:
def test_op(clean_data, fit_model):
    return dsl.ContainerOp(
        name = 'Test model',
        image = 'mavencodevv/test_ld:v.0.1',
        arguments = ['--clean_data', clean_data,
                    '--fit_model', fit_model
            ],
        file_outputs={
            'metrics': '/test_model/metrics' 
        }
    )

In [7]:
def results(metrics):
    return dsl.ContainerOp(
        name = 'Results',
        image = 'mavencodevv/metrics_ld:v.0.1',
        arguments = ['--metrics', metrics
            ] 
        
    )

In [8]:
@dsl.pipeline(
    name='Loan default pipeline',
   description='An ML reusable pipeline that predicts vehicle loan defaults'
)

# Define parameters to be fed into pipeline
def ld_pipeline(): 
    
    _load_data_op = load_data_op()
    
    _preprocess_op = preprocess_op(
        dsl.InputArgumentPath(_load_data_op.outputs['data'])).after(_load_data_op)
    
    _train_op = train_op(
        dsl.InputArgumentPath(_preprocess_op.outputs['clean_data'])).after(_preprocess_op)
    
    _test_op = test_op(
        dsl.InputArgumentPath(_preprocess_op.outputs['clean_data']),
        dsl.InputArgumentPath(_train_op.outputs['fit_model'])).after(_train_op)
    _results_op = results(
        dsl.InputArgumentPath(_test_op.outputs['metrics'])).after(_test_op)
    

In [9]:
# Compile pipeline to generate compressed YAML definition of the pipeline.
experiment_name = 'loan_default_pipeline'
#kfp.compiler.Compiler().compile(churn_reuseable_tensorflow_pipeline,  
#  '{}.zip'.format(experiment_name))
kfp.compiler.Compiler().compile(ld_pipeline, '{}.yaml'.format(experiment_name))

/root/.local/lib/python3.7/site-packages/kfp/dsl/_container_op.py:1039: FutureWarning: Please create reusable components instead of constructing ContainerOp instances directly. Reusable components are shareable, portable and have compatibility and support guarantees. Please see the documentation: https://www.kubeflow.org/docs/pipelines/sdk/component-development/#writing-your-component-definition-file The components can be created manually (or, in case of python, using kfp.components.create_component_from_func or func_to_container_op) and then loaded using kfp.components.load_component_from_file, load_component_from_uri or load_component_from_text: https://kubeflow-pipelines.readthedocs.io/en/stable/source/kfp.components.html#kfp.components.load_component_from_file
  category=FutureWarning,


In [ ]:
client = kfp.Client()
client.create_run_from_pipeline_func(ld_pipeline, arguments={})